In [1]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

# Load Data

In [2]:
df_info = pd.read_csv('Data/superhero_info.csv')
df_power = pd.read_csv('Data/superhero_powers.csv')
df_info.head(3)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"


In [3]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [4]:
df_power.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


# Split Hero and Publisher

In [5]:
df_info['Hero|Publisher'].str.split('|',expand=True)

,0,1
0,A-Bomb,Marvel Comics
1,Abe Sapien,Dark Horse Comics
2,Abin Sur,DC Comics
3,Abomination,Marvel Comics
4,Absorbing Man,Marvel Comics
...,...,...
458,Yellowjacket,Marvel Comics
459,Yellowjacket II,Marvel Comics
460,Yoda,George Lucas
461,Zatanna,DC Comics


In [6]:
df_info[['Hero','Publisher']]=df_info['Hero|Publisher'].str.split('|',expand=True)
df_info = df_info.drop(columns='Hero|Publisher')
df_info.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics


# Join the Two DFs

In [7]:
#Change name to merge on
df_power.rename(columns={"hero_names": "Hero"},inplace=True)
df_power.head()

,Hero,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [8]:
#join the two dfs
df = pd.merge(df_power,df_info, on = 'Hero')
df.head()

,Hero,Powers,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Publisher
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Marvel Comics
1,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...",Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Dark Horse Comics
2,Abin Sur,Lantern Power Ring,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",DC Comics
3,Abomination,"Accelerated Healing,Intelligence,Super Strengt...",Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Marvel Comics
4,Absorbing Man,"Cold Resistance,Durability,Energy Absorption,S...",Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Marvel Comics


# Split Height and Weight

Convert height and weight column into dictionary

In [9]:
meas = df.loc[0,'Measurements']
print(type(meas))
meas

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [10]:
meas = meas.replace("'",'"')
meas

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [11]:
fixed_meas = json.loads(meas)
print(type(fixed_meas))
fixed_meas

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [12]:
#apply to entire column
df['Measurements'] = df['Measurements'].str.replace("'",'"')
df['Measurements'] = df['Measurements'].apply(json.loads)
df['Measurements'].head(3)

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
Name: Measurements, dtype: object

In [13]:
he_we = df['Measurements'].apply(pd.Series)
he_we

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [14]:
df = pd.concat((df, he_we),axis=1)
df = df.drop(columns='Measurements')
df.head(3)

,Hero,Powers,Gender,Race,Alignment,Hair color,Eye color,Skin color,Publisher,Height,Weight
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0 cm,441.0 kg
1,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...",Male,Icthyo Sapien,good,No Hair,blue,blue,Dark Horse Comics,191.0 cm,65.0 kg
2,Abin Sur,Lantern Power Ring,Male,Ungaran,good,No Hair,blue,red,DC Comics,185.0 cm,90.0 kg


In [15]:
#Want just numerical values, so rid the values after the spaces
df['Height'] = df['Height'].str.split().str[0]
df['Weight'] = df['Weight'].str.split().str[0]
df.head(3)

,Hero,Powers,Gender,Race,Alignment,Hair color,Eye color,Skin color,Publisher,Height,Weight
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,441.0
1,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...",Male,Icthyo Sapien,good,No Hair,blue,blue,Dark Horse Comics,191.0,65.0
2,Abin Sur,Lantern Power Ring,Male,Ungaran,good,No Hair,blue,red,DC Comics,185.0,90.0


# OHE Powers

In [16]:
df_power.head()

,Hero,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [17]:
po_loc = df.loc[0,'Powers']
print(type(po_loc))
po_loc

<class 'str'>


'Accelerated Healing,Durability,Longevity,Super Strength,Stamina,Camouflage,Self-Sustenance'

In [18]:
#convert string to list
df['power_split'] = df['Powers'].str.split(",")
df['power_split'].head(3)

0    [Accelerated Healing, Durability, Longevity, S...
1    [Agility, Accelerated Healing, Cold Resistance...
2                                 [Lantern Power Ring]
Name: power_split, dtype: object

In [19]:
#create separate row for each power
exploded = df.explode('power_split')
exploded.head(3)

,Hero,Powers,Gender,Race,Alignment,Hair color,Eye color,Skin color,Publisher,Height,Weight,power_split
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,441.0,Accelerated Healing
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,441.0,Durability
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,441.0,Longevity


In [20]:
cols_to_make = exploded['power_split'].dropna().unique()
cols_to_make

array(['Accelerated Healing', 'Durability', 'Longevity', 'Super Strength',
       'Stamina', 'Camouflage', 'Self-Sustenance', 'Agility',
       'Cold Resistance', 'Underwater breathing', 'Marksmanship',
       'Weapons Master', 'Intelligence', 'Telepathy', 'Immortality',
       'Reflexes', 'Enhanced Sight', 'Sub-Mariner', 'Lantern Power Ring',
       'Super Speed', 'Invulnerability', 'Animation', 'Super Breath',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Molecular Manipulation',
       'Heat Resistance', 'Matter Absorption', 'Stealth', 'Flight',
       'Power Suit', 'Energy Blasts', 'Energy Beams', 'Power Cosmic',
       'Heat Generation', 'Danger Sense', 'Teleportation', 'Phasing',
       'Force Fields', 'Hypnokinesis', 'Energy Manipulation',
       'Invisibility', 'Enhanced Senses', 'Jump', 'Substance Secretion',
       'Natural Weapons', 'Wallcrawling', 'Vision - Thermal',
       'Power Augmentation', 'Cryokinesis', 'Dupli

In [21]:
for col in cols_to_make:
    df[col] = df['Powers'].str.contains(col)
df.head(3)

C:\Users\grace\AppData\Local\Temp\ipykernel_7420\3256936613.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = df['Powers'].str.contains(col)
C:\Users\grace\AppData\Local\Temp\ipykernel_7420\3256936613.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = df['Powers'].str.contains(col)
C:\Users\grace\AppData\Local\Temp\ipykernel_7420\3256936613.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

,Hero,Powers,Gender,Race,Alignment,Hair color,Eye color,Skin color,Publisher,Height,...,Hair Manipulation,Weather Control,Nova Force,Odin Force,Phoenix Force,Power Sense,Qwardian Power Ring,Melting,Changing Armor,Terrakinesis
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,...,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...",Male,Icthyo Sapien,good,No Hair,blue,blue,Dark Horse Comics,191.0,...,False,False,False,False,False,False,False,False,False,False
2,Abin Sur,Lantern Power Ring,Male,Ungaran,good,No Hair,blue,red,DC Comics,185.0,...,False,False,False,False,False,False,False,False,False,False


In [22]:
df = df.drop(columns=['Powers','power_split'])
df.head(3)

,Hero,Gender,Race,Alignment,Hair color,Eye color,Skin color,Publisher,Height,Weight,...,Hair Manipulation,Weather Control,Nova Force,Odin Force,Phoenix Force,Power Sense,Qwardian Power Ring,Melting,Changing Armor,Terrakinesis
0,A-Bomb,Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Male,Icthyo Sapien,good,No Hair,blue,blue,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Abin Sur,Male,Ungaran,good,No Hair,blue,red,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False


# Questions

1. Compare the average weight of super powers who have Super Speed to those who do not.

In [27]:
df['Height'] = df['Height'].astype(float)
df['Weight'] = df['Weight'].astype(float)

In [30]:
weight_group = df.groupby('Super Speed')['Weight'].mean()
print(f'The average weight of heroes with the Super Speed power is {weight_group}')

The average weight of heroes with the Super Speed power is Super Speed
False    101.773585
True     129.404040
Name: Weight, dtype: float64


2. What is the average height of heroes for each publisher?

In [32]:
height_group = df.groupby('Publisher')['Height'].mean()
height_group

Publisher
DC Comics            181.923913
Dark Horse Comics    176.909091
George Lucas         159.600000
Image Comics         211.000000
Marvel Comics        191.546128
Shueisha             171.500000
Star Trek            181.500000
Team Epic TV         180.750000
Unknown              178.000000
Name: Height, dtype: float64